Feature Engineering for Deep Learning


In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
spark = SparkSession \
    .builder \
    .appName("concrec-rank") \
    .config("spark.driver.memory", "11g") \
    .getOrCreate()

In [4]:
anime_df = spark.read.csv('../dataset/parsed_anime.csv', header=True, inferSchema=True)


In [5]:
anime_df.printSchema()


root
 |-- anime_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- type: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- members: integer (nullable = true)
 |-- japanese_title: string (nullable = true)
 |-- aired: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- aired_from: string (nullable = true)
 |-- aired_to: integer (nullable = true)



In [6]:
from pyspark.sql.types import IntegerType
anime_df = anime_df.withColumn('aired_from', col('aired_from').cast('int'))

In [7]:
anime_df.show(5)


+--------+--------------------+--------------------+-----+--------+------+-------+--------------------------+--------------------+--------------------+----------+----------+
|anime_id|                name|               genre| type|episodes|rating|members|            japanese_title|               aired|           image_url|aired_from|  aired_to|
+--------+--------------------+--------------------+-----+--------+------+-------+--------------------------+--------------------+--------------------+----------+----------+
|   32281|      Kimi no Na wa.|Drama, Romance, S...|Movie|       1|  9.37| 200630|                君の名は。|        Aug 26, 2016|https://cdn.myani...|1472140800|1472140800|
|    5114|Fullmetal Alchemi...|Action, Adventure...|   TV|      64|  9.26| 793665|鋼の錬金術師 FULLMETAL ...|Apr 5, 2009 to Ju...|https://cdn.myani...|1238860800|1278172800|
|   28977|            Gintama°|Action, Comedy, H...|   TV|      51|  9.25| 114262|                     銀魂°|Apr 8, 2015 to Ma...|https://cdn.m

In [8]:
rating_df = spark.read.csv('../dataset/rating.csv', header=True, inferSchema=True)


In [9]:
rating_df = rating_df.filter(rating_df['rating'] > 0)


In [10]:
merged_df = rating_df.join(
    anime_df.select('anime_id', 'name', 'genre', 'type', 'episodes', 
                    'rating', 'members', 'aired_from', 'aired_to').withColumnRenamed('rating', 'all_rating'), 
    on=['anime_id'], how='left'
)

In [11]:
merged_df.show(5)


+--------+-------+------+--------------------+--------------------+----+--------+----------+-------+----------+----------+
|anime_id|user_id|rating|                name|               genre|type|episodes|all_rating|members|aired_from|  aired_to|
+--------+-------+------+--------------------+--------------------+----+--------+----------+-------+----------+----------+
|    8074|      1|    10|Highschool of the...|Action, Ecchi, Ho...|  TV|      12|      7.46| 535892|1278259200|1284912000|
|   11617|      1|    10|     High School DxD|Comedy, Demons, E...|  TV|      12|       7.7| 398660|1325779200|1332432000|
|   11757|      1|    10|    Sword Art Online|Action, Adventure...|  TV|      25|      7.83| 893100|1341676800|1356192000|
|   15451|      1|    10| High School DxD New|Action, Comedy, D...|  TV|      12|      7.87| 266657|1373126400|1379779200|
|   11771|      2|    10|    Kuroko no Basket|Comedy, School, S...|  TV|      25|      8.46| 338315|1333814400|1348243200|
+--------+------

### Build Label


In [12]:
like_threshold = 7.5

def build_label(df):
    return df.withColumn('label',
                         when(col('rating') >= like_threshold, 1).otherwise(0)
                        )

In [13]:
labeled_df = build_label(merged_df)
labeled_df.show(5)

+--------+-------+------+--------------------+--------------------+----+--------+----------+-------+----------+----------+-----+
|anime_id|user_id|rating|                name|               genre|type|episodes|all_rating|members|aired_from|  aired_to|label|
+--------+-------+------+--------------------+--------------------+----+--------+----------+-------+----------+----------+-----+
|    8074|      1|    10|Highschool of the...|Action, Ecchi, Ho...|  TV|      12|      7.46| 535892|1278259200|1284912000|    1|
|   11617|      1|    10|     High School DxD|Comedy, Demons, E...|  TV|      12|       7.7| 398660|1325779200|1332432000|    1|
|   11757|      1|    10|    Sword Art Online|Action, Adventure...|  TV|      25|      7.83| 893100|1341676800|1356192000|    1|
|   15451|      1|    10| High School DxD New|Action, Comedy, D...|  TV|      12|      7.87| 266657|1373126400|1379779200|    1|
|   11771|      2|    10|    Kuroko no Basket|Comedy, School, S...|  TV|      25|      8.46| 3383

### Sliding Window
这里要在df的每个row上，额外增加和用户相关的信息。比如该用户最爱的电影类型、该用户看过多少电影、平均打分是多少
为了防止泄露未来信息，需把所有评分按照时间顺序排序，然后用滑动窗口聚合
理论应该使用评分时间，但是由于没有这个数据，所以采用电影上映时间

In [14]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import pyspark.sql.types as types

In [15]:
windowSpec = Window \
    .partitionBy('user_id') \
    .orderBy('aired_from') \
    .rowsBetween(-100, -1)

In [16]:
# 帮助方法：对于某一列，在聚合的时候，如果用户不喜欢这个电影，则不聚合这个电影的信息
likedMoviesCol = lambda cname: when(col('label') == 1, col(cname)).otherwise(lit(None))

@udf(types.ArrayType(types.StringType()))
def most_liked_genres(gen_strs):
    """
    gen_strs = ["Action, Adventure, Drama", "Comedy, Drama, School"]
    """
    gens = [s.split(",") for s in gen_strs]
    gens = [x for l in gens for x in l] # flatten
    gens = [s.strip() for s in gens]
    
    gen_set = set(gens)
    count_occur = lambda gen, l: len([g for g in l if g == gen])
    gen_with_occur = [(gen, count_occur(gen, gens)) for gen in gen_set]
    gen_with_occur.sort(key=lambda x: x[1], reverse=True)
    
    # pick 3 most liked genres
    return [x[0] for x in gen_with_occur[:5]]

In [18]:
# 帮助方法：对于某一列，在聚合的时候，如果用户不喜欢这个电影，则不聚合这个电影的信息
likedMoviesCol = lambda colname: when(col('label') == 1, col(colname)).otherwise(lit(None))


@udf(returnType='array<string>')
def genre_to_list(gen_str):
    if gen_str is None:
        return []
    
    gens = gen_str.split(",")
    return [gen.strip() for gen in gens]


@udf(types.ArrayType(types.StringType()))
def most_liked_genres(gen_strs):
    """
    gen_strs = ["Action, Adventure, Drama", "Comedy, Drama, School"]
    """
    gens = [s.split(",") for s in gen_strs]
    gens = [x for l in gens for x in l] # flatten
    gens = [s.strip() for s in gens]
    
    gen_set = set(gens)
    count_occur = lambda gen, l: len([g for g in l if g == gen])
    gen_with_occur = [(gen, count_occur(gen, gens)) for gen in gen_set]
    gen_with_occur.sort(key=lambda x: x[1], reverse=True)
    
    # pick 5 most liked genres
    return [x[0] for x in gen_with_occur[:5]]

In [19]:
feat_df = merged_df \
    .withColumn('genres', genre_to_list(col('genre'))) \
    .withColumn('user_rating_cnt', count(lit(1)).over(windowSpec)) \
    .withColumn('user_rating_ave', mean(col('rating')).over(windowSpec)) \
    .withColumn('user_rating_ave', F.round(col('user_rating_ave'), NUMBER_PRECISION)) \
    .withColumn('user_rating_std', stddev(col('rating')).over(windowSpec)) \
    .withColumn('user_rating_std', F.round(col('user_rating_std'), NUMBER_PRECISION)) \
    .withColumn('user_aired_from_ave', mean(likedMoviesCol('aired_from')).over(windowSpec)) \
    .withColumn('user_aired_from_ave', F.round(col('user_aired_from_ave'), 0)) \
    .withColumn('user_aired_to_ave', mean(likedMoviesCol('aired_to')).over(windowSpec)) \
    .withColumn('user_aired_to_ave', F.round(col('user_aired_to_ave'), 0)) \
    .withColumn('user_liked_genres', most_liked_genres(collect_list(likedMoviesCol('genre')).over(windowSpec)))

NameError: name 'NUMBER_PRECISION' is not defined